# Twitter through Tweepy

In [4]:
import tweepy
import re
import pandas as pd
from tabulate import tabulate
from datetime import datetime, timedelta
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Twitter API connection
API_KEY = "5EXwzdy71UuO6ivnQmAe7IvpJ"
API_SECRET_KEY = "CaPwjZUR370mPhoBrL8luE0l31r5Kl4tYA0ja56hSBKrchVx9h"
ACCESS_TOKEN = "932296106456862721-gaKleWHmBTBRXs2dct2iYRlKq3Lxcr8"
ACCESS_TOKEN_SECRET = "DM7O5SKNZhuAyYYZOcmBhyt9xt0TEffdlmwbLIIihwT9K"

# Authenticate with Twitter API
auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Initialize VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

def fetch_twitter_data(query):
    """Fetches tweets based on a query for the last 7 days."""
    data = []
    seven_days_ago = datetime.utcnow() - timedelta(days=7)  # Fetch tweets from the last 7 days

    for tweet in tweepy.Cursor(api.search_tweets, q=query, lang="en", tweet_mode="extended").items(500):  # Adjust the limit as needed
        tweet_datetime = tweet.created_at
        if tweet_datetime >= seven_days_ago:  # Include tweets only from the last 7 days
            tweet_data = {
                'Tweet': tweet.full_text,
                'DateTime': tweet_datetime.strftime('%Y-%m-%d %H:%M:%S')  # Format as YYYY-MM-DD HH:MM:SS
            }
            data.append(tweet_data)
    return pd.DataFrame(data)

def sentiment_score(text):
    """Calculates sentiment polarity using VADER."""
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']  # Compound score is the overall sentiment

def find_coin_mentions(text):
    """Identifies potential cryptocurrency symbols."""
    pattern = r"\$[A-Z]+"  # Matches symbols like $BTC, $ETH
    return re.findall(pattern, text)

def analyze_positive_sentiment(data):
    """Analyzes data for coin mentions with positive sentiment."""
    data['Sentiment'] = data['Tweet'].apply(sentiment_score)
    data['Coin Mentions'] = data['Tweet'].apply(find_coin_mentions)

    # Explode coin mentions
    exploded_data = data.explode('Coin Mentions').dropna(subset=['Coin Mentions'])

    # Filter for stronger positive sentiments (Sentiment >= 0.2)
    positive_data = exploded_data[exploded_data['Sentiment'] >= 0.2]

    # Group by Coin Mentions to get the mentions count and the average sentiment
    positive_summary = positive_data.groupby('Coin Mentions').agg(
        Count=('Coin Mentions', 'size'),
        Average_Positive_Sentiment=('Sentiment', 'mean')
    ).reset_index()

    # Sort by count of mentions (most positively mentioned first)
    positive_summary = positive_summary.sort_values(by='Count', ascending=False)

    # Get the top 10 most positively mentioned coins
    top_10_positive = positive_summary.head(10)

    return top_10_positive

def main():
    query = "#crypto OR #Solana OR #memeCoin"  # Specify your search query

    print("Fetching Twitter data from the last 7 days...")
    data = fetch_twitter_data(query)

    if data.empty:
        print("No data found for the last 7 days.")
        return

    print("Analyzing data for positive sentiment...")
    positive_summary = analyze_positive_sentiment(data)

    if positive_summary.empty:
        print("No coins with positive sentiment found.")
    else:
        # Display results in a table format
        print("\nTop 10 Coins with Positive Sentiment (Last 7 Days):")
        print(tabulate(
            positive_summary,
            headers=['Coin', 'Positive Mentions', 'Avg Positive Sentiment'],
            tablefmt='fancy_grid'
        ))

# ✅ FIXED: Corrected __name__ check
if __name__ == "__main__":
    main()


Fetching Twitter data from the last 7 days...


Forbidden: 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product

In [2]:
pip install tweepy

  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import tweepy
import pandas as pd
from datetime import datetime, timedelta
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Twitter API v2 Credentials
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAOwyygEAAAAARtRq58I0mXt%2FHuBNRyiOneTSkiU%3DI26Cwxp9mx4C6nKM8MV9VcQfXFdOlviFceyNqa0H5YxWlvEa7C"  # Replace with the correct token

# Authenticate using Tweepy v2
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Initialize VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

def fetch_twitter_data(query):
    """Fetches tweets based on a query for the last 7 days using Twitter API v2."""
    data = []
    
    # Get the time 7 days ago and ensure it's in ISO format (UTC)
    seven_days_ago = datetime.utcnow() - timedelta(days=7)
    seven_days_ago = seven_days_ago.replace(microsecond=0).isoformat() + "Z"  # Format as UTC with "Z"

    print(f"Fetching tweets for the query: '{query}' since {seven_days_ago}...")

    try:
        # Search tweets from the last 7 days
        tweets = client.search_recent_tweets(
            query=query, 
            max_results=100,  # Max per request is 100
            tweet_fields=["created_at", "text"],
            start_time=seven_days_ago  # Ensure we limit to the last 7 days
        )

        if tweets.data:
            for tweet in tweets.data:
                tweet_data = {
                    "Tweet": tweet.text,
                    "DateTime": tweet.created_at.strftime('%Y-%m-%d %H:%M:%S')
                }
                data.append(tweet_data)
            print(f"Found {len(tweets.data)} tweets.")
        else:
            print("No tweets found for this query in the last 7 days.")
    except tweepy.TweepyException as e:
        print("Error fetching tweets:", e)

    return pd.DataFrame(data)

def sentiment_score(text):
    """Calculates sentiment polarity using VADER."""
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']  # Compound score is the overall sentiment

def find_coin_mentions(text):
    """Identifies potential cryptocurrency symbols."""
    pattern = r"\$[A-Z]+"  # Matches symbols like $BTC, $ETH
    return re.findall(pattern, text)

def analyze_positive_sentiment(data):
    """Analyzes data for coin mentions with positive sentiment."""
    if data.empty:
        print("No data to analyze.")
        return pd.DataFrame()

    data['Sentiment'] = data['Tweet'].apply(sentiment_score)
    data['Coin Mentions'] = data['Tweet'].apply(find_coin_mentions)

    # Explode coin mentions
    exploded_data = data.explode('Coin Mentions').dropna(subset=['Coin Mentions'])

    # Filter for stronger positive sentiments (Sentiment >= 0.2)
    positive_data = exploded_data[exploded_data['Sentiment'] >= 0.2]

    # Group by Coin Mentions to get the mentions count and the average sentiment
    positive_summary = positive_data.groupby('Coin Mentions').agg(
        Count=('Coin Mentions', 'size'),
        Average_Positive_Sentiment=('Sentiment', 'mean')
    ).reset_index()

    return positive_summary

def main():
    # Refined search query to focus on Solana and MemeCoins
    query = "#Solana OR #memeCoin OR $SOL OR $MEME"

    print("Fetching Twitter data from the last 7 days...")
    data = fetch_twitter_data(query)

    if data.empty:
        print("No data found for the last 7 days.")
        return

    print("Analyzing data for positive sentiment...")
    positive_summary = analyze_positive_sentiment(data)

    if positive_summary.empty:
        print("No coins with positive sentiment found.")
    else:
        # Display results in a table format
        print("\nTop 10 Coins with Positive Sentiment (Last 7 Days):")
        print(tabulate(
            positive_summary.head(10),
            headers=['Coin', 'Positive Mentions', 'Avg Positive Sentiment'],
            tablefmt='fancy_grid'
        ))

if __name__ == "__main__":
    main()


Fetching Twitter data from the last 7 days...
Fetching tweets for the query: '#Solana OR #memeCoin OR $SOL OR $MEME' since 2025-01-22T08:30:07Z...
Error fetching tweets: 429 Too Many Requests
Too Many Requests
No data found for the last 7 days.


In [6]:
import tweepy
import pandas as pd
from datetime import datetime, timedelta
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time

# Twitter API v2 Credentials
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAOwyygEAAAAARtRq58I0mXt%2FHuBNRyiOneTSkiU%3DI26Cwxp9mx4C6nKM8MV9VcQfXFdOlviFceyNqa0H5YxWlvEa7C"  # Replace with the correct token

# Authenticate using Tweepy v2
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Initialize VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

def fetch_twitter_data(query):
    """Fetches tweets based on a query for the last 7 days using Twitter API v2."""
    data = []
    
    # Get the time 7 days ago and ensure it's in ISO format (UTC)
    seven_days_ago = datetime.utcnow() - timedelta(days=7)
    seven_days_ago = seven_days_ago.replace(microsecond=0).isoformat() + "Z"  # Format as UTC with "Z"

    print(f"Fetching tweets for the query: '{query}' since {seven_days_ago}...")

    try:
        # Search tweets from the last 7 days
        tweets = client.search_recent_tweets(
            query=query, 
            max_results=100,  # Max per request is 100
            tweet_fields=["created_at", "text"],
            start_time=seven_days_ago  # Ensure we limit to the last 7 days
        )

        if tweets.data:
            for tweet in tweets.data:
                tweet_data = {
                    "Tweet": tweet.text,
                    "DateTime": tweet.created_at.strftime('%Y-%m-%d %H:%M:%S')
                }
                data.append(tweet_data)
            print(f"Found {len(tweets.data)} tweets.")
        else:
            print("No tweets found for this query in the last 7 days.")
    except tweepy.TooManyRequests as e:
        print("Rate limit exceeded. Waiting for reset...")
        # Get the time to wait for rate limit reset from the exception's response
        reset_time = e.response.headers.get('x-rate-limit-reset')
        if reset_time:
            reset_time = int(reset_time)
            current_time = int(time.time())
            wait_time = reset_time - current_time
            print(f"Waiting for {wait_time} seconds.")
            time.sleep(wait_time + 5)  # Adding a small buffer time
            # Retry fetching the data after waiting
            return fetch_twitter_data(query)
        else:
            print("Unable to determine when the rate limit will reset.")
    
    except tweepy.TweepyException as e:
        print("Error fetching tweets:", e)

    return pd.DataFrame(data)

def sentiment_score(text):
    """Calculates sentiment polarity using VADER."""
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']  # Compound score is the overall sentiment

def find_coin_mentions(text):
    """Identifies potential cryptocurrency symbols."""
    pattern = r"\$[A-Z]+"  # Matches symbols like $BTC, $ETH
    return re.findall(pattern, text)

def analyze_positive_sentiment(data):
    """Analyzes data for coin mentions with positive sentiment."""
    if data.empty:
        print("No data to analyze.")
        return pd.DataFrame()

    data['Sentiment'] = data['Tweet'].apply(sentiment_score)
    data['Coin Mentions'] = data['Tweet'].apply(find_coin_mentions)

    # Explode coin mentions
    exploded_data = data.explode('Coin Mentions').dropna(subset=['Coin Mentions'])

    # Filter for stronger positive sentiments (Sentiment >= 0.2)
    positive_data = exploded_data[exploded_data['Sentiment'] >= 0.2]

    # Group by Coin Mentions to get the mentions count and the average sentiment
    positive_summary = positive_data.groupby('Coin Mentions').agg(
        Count=('Coin Mentions', 'size'),
        Average_Positive_Sentiment=('Sentiment', 'mean')
    ).reset_index()

    return positive_summary

def main():
    # Refined search query to focus on Solana and MemeCoins
    query = "#Solana OR #memeCoin OR $SOL OR $MEME"

    print("Fetching Twitter data from the last 7 days...")
    data = fetch_twitter_data(query)

    if data.empty:
        print("No data found for the last 7 days.")
        return

    print("Analyzing data for positive sentiment...")
    positive_summary = analyze_positive_sentiment(data)

    if positive_summary.empty:
        print("No coins with positive sentiment found.")
    else:
        # Display results in a table format
        print("\nTop 10 Coins with Positive Sentiment (Last 7 Days):")
        print(tabulate(
            positive_summary.head(10),
            headers=['Coin', 'Positive Mentions', 'Avg Positive Sentiment'],
            tablefmt='fancy_grid'
        ))

if __name__ == "__main__":
    main()


Fetching Twitter data from the last 7 days...
Fetching tweets for the query: '#Solana OR #memeCoin OR $SOL OR $MEME' since 2025-01-22T08:34:21Z...
Rate limit exceeded. Waiting for reset...
Waiting for 526 seconds.
Fetching tweets for the query: '#Solana OR #memeCoin OR $SOL OR $MEME' since 2025-01-22T08:43:12Z...
Error fetching tweets: 400 Bad Request
There were errors processing your request: Reference to invalid operator 'cashtag'. Operator is not available in current product or product packaging. Please refer to complete available operator list at http://t.co/operators. (at position 25), Reference to invalid operator 'cashtag'. Operator is not available in current product or product packaging. Please refer to complete available operator list at http://t.co/operators. (at position 33)
No data found for the last 7 days.


In [7]:
pip install snscrape

Note: you may need to restart the kernel to use updated packages.


In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import datetime, timedelta
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re

# Initialize VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

# Define the query (hashtags and $ symbols for specific coins)
query = '#Solana OR #memeCoin OR $SOL OR $MEME'

# Define the time range (last 7 days)
end_date = datetime.utcnow()
start_date = end_date - timedelta(days=7)

# Function to scrape tweets from the last 7 days
def fetch_twitter_data(query, start_date, end_date):
    """Fetch tweets using snscrape for the last 7 days."""
    data = []

    # Using snscrape to fetch tweets
    for tweet in sntwitter.TwitterSearchScraper(f'{query} since:{start_date.strftime("%Y-%m-%d")} until:{end_date.strftime("%Y-%m-%d")}').get_items():
        tweet_data = {
            'Tweet': tweet.content,
            'DateTime': tweet.date.strftime('%Y-%m-%d %H:%M:%S')
        }
        data.append(tweet_data)
    
    return pd.DataFrame(data)

# Fetch data
print("Fetching Twitter data from the last 7 days...")
data = fetch_twitter_data(query, start_date, end_date)

if data.empty:
    print("No data found for the last 7 days.")
else:
    print(f"Found {len(data)} tweets in the last 7 days.")

# Function to calculate sentiment scores using VADER
def sentiment_score(text):
    """Calculates sentiment polarity using VADER."""
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']  # Compound score is the overall sentiment

# Function to find coin mentions in the tweet text
def find_coin_mentions(text):
    """Identifies potential cryptocurrency symbols."""
    pattern = r"\$[A-Z]+"  # Matches symbols like $BTC, $ETH
    return re.findall(pattern, text)

# Analyze positive sentiment for coin mentions
def analyze_positive_sentiment(data):
    """Analyzes data for coin mentions with positive sentiment."""
    if data.empty:
        print("No data to analyze.")
        return pd.DataFrame()

    data['Sentiment'] = data['Tweet'].apply(sentiment_score)
    data['Coin Mentions'] = data['Tweet'].apply(find_coin_mentions)

    # Explode coin mentions
    exploded_data = data.explode('Coin Mentions').dropna(subset=['Coin Mentions'])

    # Filter for stronger positive sentiments (Sentiment >= 0.2)
    positive_data = exploded_data[exploded_data['Sentiment'] >= 0.2]

    # Group by Coin Mentions to get the mentions count and the average sentiment
    positive_summary = positive_data.groupby('Coin Mentions').agg(
        Count=('Coin Mentions', 'size'),
        Average_Positive_Sentiment=('Sentiment', 'mean')
    ).reset_index()

    return positive_summary

# Analyzing the data for positive sentiment
print("Analyzing data for positive sentiment...")
positive_summary = analyze_positive_sentiment(data)

if positive_summary.empty:
    print("No coins with positive sentiment found.")
else:
    # Display results in a table format
    from tabulate import tabulate
    print("\nTop 10 Coins with Positive Sentiment (Last 7 Days):")
    print(tabulate(
        positive_summary.head(10),
        headers=['Coin', 'Positive Mentions', 'Avg Positive Sentiment'],
        tablefmt='fancy_grid'
    ))


Fetching Twitter data from the last 7 days...


Error retrieving https://twitter.com/search?f=live&lang=en&q=%23Solana+OR+%23memeCoin+OR+%24SOL+OR+%24MEME+since%3A2025-01-22+until%3A2025-01-29&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%23Solana+OR+%23memeCoin+OR+%24SOL+OR+%24MEME+since%3A2025-01-22+until%3A2025-01-29&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1125)')))"))
4 requests to https://twitter.com/search?f=live&lang=en&q=%23Solana+OR+%23memeCoin+OR+%24SOL+OR+%24MEME+since%3A2025-01-22+until%3A2025-01-29&src=spelling_expansion_revert_click failed, giving up.
Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%23Solana+OR+%23memeCoin+OR+%24SOL+OR+%24MEME+since%3A2025-01-

ScraperException: 4 requests to https://twitter.com/search?f=live&lang=en&q=%23Solana+OR+%23memeCoin+OR+%24SOL+OR+%24MEME+since%3A2025-01-22+until%3A2025-01-29&src=spelling_expansion_revert_click failed, giving up.

In [10]:
conda install openssl

Solving environment: ...working... done

# All requested packages already installed.

Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 25.1.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [1]:
conda install certifi

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 25.1.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [2]:
conda update -n base -c defaults conda

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Maham-Hadia\anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-token-0.5.0          |     pyhd3eb1b0_0          11 KB
    xmltodict-0.13.0           |   py38haa95532_0          19 KB
    ------------------------------------------------------------
                                           Total:          30 KB

The following packages will be UPDATED:

  conda-token                            0.3.0-pyhd3eb1b0_0 --> 0.5.0-pyhd3eb1b0_0
  xmltodict          pkgs/main/noarch::xmltodict-0.12.0-py~ --> pkgs/main/win-64::xmltodict-0.13.0-py38haa95532_0




conda-token-0.5.0    | 11 KB     |            |   0% 
conda-token-0.5.0    | 11 KB     | ########## | 100% 
conda-token-0.5.0    | 11 KB     | ########## | 100% 

xmltodict-0.13.0     | 19 KB     |



==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 25.1.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re

# Initialize VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

# Define the query (hashtags and $ symbols for specific coins)
query = '#Solana OR #memeCoin OR $SOL OR $MEME'

# Function to scrape tweets (without time range, to get the latest trending tweets)
def fetch_twitter_data(query):
    """Fetch tweets using snscrape."""
    data = []

    # Using snscrape to fetch tweets (no time range)
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        tweet_data = {
            'Tweet': tweet.content,
            'DateTime': tweet.date.strftime('%Y-%m-%d %H:%M:%S')
        }
        data.append(tweet_data)

    return pd.DataFrame(data)

# Fetch data
print("Fetching Twitter data for the latest trends...")
data = fetch_twitter_data(query)

if data.empty:
    print("No data found.")
else:
    print(f"Found {len(data)} tweets.")

# Function to calculate sentiment scores using VADER
def sentiment_score(text):
    """Calculates sentiment polarity using VADER."""
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']  # Compound score is the overall sentiment

# Function to find coin mentions in the tweet text
def find_coin_mentions(text):
    """Identifies potential cryptocurrency symbols."""
    pattern = r"\$[A-Z]+"  # Matches symbols like $BTC, $ETH
    return re.findall(pattern, text)

# Analyze positive sentiment for coin mentions
def analyze_positive_sentiment(data):
    """Analyzes data for coin mentions with positive sentiment."""
    if data.empty:
        print("No data to analyze.")
        return pd.DataFrame()

    data['Sentiment'] = data['Tweet'].apply(sentiment_score)
    data['Coin Mentions'] = data['Tweet'].apply(find_coin_mentions)

    # Explode coin mentions
    exploded_data = data.explode('Coin Mentions').dropna(subset=['Coin Mentions'])

    # Filter for stronger positive sentiments (Sentiment >= 0.2)
    positive_data = exploded_data[exploded_data['Sentiment'] >= 0.2]

    # Group by Coin Mentions to get the mentions count and the average sentiment
    positive_summary = positive_data.groupby('Coin Mentions').agg(
        Count=('Coin Mentions', 'size'),
        Average_Positive_Sentiment=('Sentiment', 'mean')
    ).reset_index()

    return positive_summary

# Analyzing the data for positive sentiment
print("Analyzing data for positive sentiment...")
positive_summary = analyze_positive_sentiment(data)

if positive_summary.empty:
    print("No coins with positive sentiment found.")
else:
    # Display results in a table format
    from tabulate import tabulate
    print("\nTop Coins with Positive Sentiment (Trending):")
    print(tabulate(
        positive_summary.head(10),  # Display top 10 most positively mentioned coins
        headers=['Coin', 'Positive Mentions', 'Avg Positive Sentiment'],
        tablefmt='fancy_grid'
    ))


Fetching Twitter data for the latest trends...


Error retrieving https://twitter.com/search?f=live&lang=en&q=%23Solana+OR+%23memeCoin+OR+%24SOL+OR+%24MEME&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%23Solana+OR+%23memeCoin+OR+%24SOL+OR+%24MEME&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))"))
4 requests to https://twitter.com/search?f=live&lang=en&q=%23Solana+OR+%23memeCoin+OR+%24SOL+OR+%24MEME&src=spelling_expansion_revert_click failed, giving up.
Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%23Solana+OR+%23memeCoin+OR+%24SOL+OR+%24MEME&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate 

ScraperException: 4 requests to https://twitter.com/search?f=live&lang=en&q=%23Solana+OR+%23memeCoin+OR+%24SOL+OR+%24MEME&src=spelling_expansion_revert_click failed, giving up.

In [3]:
pip install --upgrade snscrape

Note: you may need to restart the kernel to use updated packages.
